#  Assignment 5

## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|. | | |
|  | |. |

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


We maken de toetsassignment af. We bekijken dus weer het artikel <http://language.media.mit.edu/paper>. In de toets hebben we de netwerkdata gebaseerd op boekvertalingen gemaakt.
In dit huiswerk gaan we verder.

 

In [2]:
%matplotlib inline
import pandas as pd
import re
from lxml import etree 
from bz2file import BZ2File
import codecs
import nltk
from collections import defaultdict
from  itertools import combinations  # ideal for creating all possible pairs that out can make out of a set
from __future__ import division
from math import sqrt

### Deel  Wikipedia  

 

1. Maak   van de Wikipedia _raw_ dataset de netwerk  datasets die op <http://language.media.mit.edu/data> beschreven staan. 
2. We bekijken in hoeverre onze resultaten afwijken van die in de geleverde bewerkte datasets.
3. We leggen eventuele verschillen uit.

#### In Stapjes

We doen de opdracht in een aantal stapjes.

#### Let op, je notebook moet vanzelf draaien. Alle data moet opgehaald worden.

# Download 

1. Download de ruwe dataset als zip file, ontzip hem en sla hem in de directory op waarin dit notebook ook staat.
    * Dit mag je met de hand doen. De code **moet niet runbaar** in je notebook staan.
    * Maar zorg dat de rest van de code er vanuit gaat dat `wikipedia_userlang_iso639-3.tsv` in dezelfde folder staat als dit notebook
2. Lees de 2 bewerkte datasets over Wikipedia _Wikipedia Stats by Language_ en _Edges_ direct in van de urls in  pandas Dataframes, die je respectivelijk `dataset_stats_wikipedia` en `wikipedia_edges` noemt.
    1. Maak de Taal code de index in het eerste frame, en hou alleen de Kolommen `['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']`
    2. Sorteer het frame op de index
    2. Verwijder de kolommen `SourceLanguageName	TargetLanguageName	SourcePopulation	TargetPopulations` uit het tweede frame
    3. Sorteer het frame op `SourceLanguageName`
    3. Laat steeds met `head()` zien dat het werkt.
    
### Je antwoord bestaat uit 2 cellen

In [151]:
#1
#wikipedia_userlang = pd.read_csv('wikipedia_userlang_iso639-3.tsv', sep = '\t')

#2
dataset_stats_wikipedia = pd.read_csv('http://language.media.mit.edu/data/public/dataset_stats_wikipedia.tsv', sep = '\t')
dataset_stats_wikipedia.head()

,Language,Code,Edits,Editors,Avg. edits per editor,Edits by mulitlinguals,Multilinguals,Avg. edits per multilingual editor,Total editors,Total multilinguals
0,Abkhazian,abk,2807,15,187.133333,1494,7,213.428571,2562860,279172
1,Afar,aar,46,4,11.500000,9,3,3.000000,2562860,279172
2,Afrikaans,afr,103652,511,202.841487,95988,352,272.693182,2562860,279172
3,Akan,aka,327,31,10.548387,240,24,10.000000,2562860,279172
4,Albanian,sqi,196685,1996,98.539579,137800,1083,127.239150,2562860,279172


In [40]:
#2C & 2D
wikipedia_edges2 = wikipedia_edges.drop(['SourceLanguageName', 'TargetLanguageName', 'SourcePopulation', 'TargetPopulations'], axis=1).sort_values(by='SourceLanguageCode', ascending=True)

#2E
wikipedia_edges2.head()

,SourceLanguageCode,TargetLanguageCode,Coocurrences,PhiCorrelation,Tstatistic
0,abk,rus,106,0.026996,8.599220
1,abq,rus,8,0.008546,2.721204
2,ady,rus,71,0.027903,8.888263
3,afr,eng,121,0.013972,15.467151
4,afr,nld,108,0.014675,4.897763


# Haal gegevens uit de ruwe data, deel 1

* De file `'wikipedia_userlang_iso639-3.tsv'` lijkt een spreadsheet maar hij heeft niet op elke regel evenveel kolommen.
* Processing met pandas is dus ook lastig.
* Als alternatief gaan we file zelf regel voor regel  inlezen, en de gewenste gegevens opslaan.
* Ons einddoel is om een dataframe te maken dat er precies zo uitziet als `dataset_stats_wikipedia`
* We doen dat in 2 stappen:
    1. We maken een aantal dicts aan met de gewenste gegevens
    2. Van elke dict maken we een dataframe met `pd.DataFrame.from_dict`
    3. Die plakken we allemaal aan elkaar met `join` (neem een inner join)
    4. Noem het eindresultaat `stats`, sorteer het ook op de index en laat het begin zien met `head`.
    5. Check natuurlijk zelf of je op de goede weg zit door je resultaat te vergelijken met `dataset_stats_wikipedia`
* Hieronder staat wat code om je op weg te helpen
* **Lees het artikel en de supplementary material goed om net als zij de ruwe data te filteren.**


# Haal gegevens uit de ruwe data, deel 2

* In dezelfde code waarin je de vorige vraag hebt gemaakt, maak je ook een dict `pairs` aan. Hier zijn de sleutels paren van taalcodes, **waarbij de eerste altijd alfabetisch eerder is dan de tweede**, en hun waarde het aantal cooccurences.
* Maak van dit dict een Dataframe `edges`, waarbij `edges.columns=['SourceLanguageCode','TargetLanguageCode','Coocurrencesmy']`
* Sorteer ook hier op `'SourceLanguageCode'`
* **Lees het artikel en de supplementary material goed om net als zij de ruwe data te filteren.**

In [65]:
with open('wikipedia_userlang_iso639-3.tsv') as f:    
    c=0
    editors=defaultdict(int)
    edits=defaultdict(int)
    multilingual=defaultdict(int)
    Edits_by_multilinguals=defaultdict(int)
    pairs= defaultdict(int)
    for l in f:
        c += 1
        totaled = 0
        line = l.split('\t')[1:]
        cells = []
        for cell in line:
            cell = cell.split(',')
            if len(cell) == 2:
                totaled += int(cell[1])
                cells.append(cell)
        if totaled >= 5 and len(cells) <= 5:
            for cell in line:
                cell = cell.split(',')
                if len(cell) == 2 and cell[1]>=2:
                    editors[cell[0]] += 1
                    edits[cell[0]] += int(cell[1])
            #Add to the multilingual dict and edit by multilingual
            if len(cells) > 1:
                combi = []
                for x in cells:
                    combi.append(x[0])                
                    multilingual[x[0]] += 1
                    Edits_by_multilinguals[x[0]] += int(x[1])
                combi = list(combinations(sorted(combi), 2))
                for pair in combi:
                    if pair[0]<pair[1]:
                         pairs[pair] += 1

#        if c == 2000:
#            print pairs
#            break

In [133]:
#Deel 1
#Aanmaken Dataframe editors
editorsdf = pd.DataFrame.from_dict(editors, orient='index')
editorsdf.columns = ['Editors']

#Aanmaken Dataframe edits
editsdf = pd.DataFrame.from_dict(edits, orient='index')
editsdf.columns = ['Edits']

#Aanmaken Dataframe multilingual
multilingualdf = pd.DataFrame.from_dict(multilingual, orient='index')
multilingualdf.columns = ['Multilingual']

#Aanmaken Dataframe Edits_by_mulitlinguals
Edits_by_multilingualsdf = pd.DataFrame.from_dict(Edits_by_multilinguals, orient='index')
Edits_by_multilingualsdf.columns = ['Edits by multilinguals']

stats = pd.concat([editorsdf, editsdf, multilingualdf, Edits_by_multilingualsdf], join='inner',axis=1)
stats.sort_index().head()

,Editors,Edits,Multilingual,Edits by multilinguals
aar,8,51,8,51
abk,23,2815,17,1540
afr,730,103893,600,97942
aka,49,346,43,279
amh,88,4619,65,4062


In [150]:
#Deel2
#Aanmaken Dataframe Edges
edges = pd.DataFrame(pairs.values(), index=pd.MultiIndex.from_tuples(pairs.keys())).sort_index().reset_index()
edges.columns=['SourceLanguageCode','TargetLanguageCode','Coocurrencesmy']
edges.head(10)

,SourceLanguageCode,TargetLanguageCode,Coocurrencesmy
0,aar,arg,1
1,aar,bis,1
2,aar,deu,3
3,aar,eng,2
4,aar,fin,1
5,aar,ipk,1
6,aar,ita,3
7,aar,kat,1
8,aar,kor,1
9,aar,lao,1


# Vergelijk 

We gaan nu onze dataframes vergelijken met die we hebben opgehaald.

We zijn natuurlijk erg benieuwd of er verschillen zijn, en vooral ook groot die dan zijn, als ze er zijn.

1. Join `dataset_stats_wikipedia` en `stats` in een nieuwe dataframe `samen`. 
    * laat met `samen.describe()` zien hoe groot of klein de verschillen zijn
2. Maak een functie `fout_percentage` waar je twee kolommen en een dataframe aan geeft en die _het verschil tussen die twee kolommen bepaalt, en omzet naar een "foutpercentage"_.
3. Bereken dat percentage voor de vier kolommen `['Editors', 'Edits','Multilinguals','Edits by mulitlinguals']` en print het netjes uit.
4. Leg in een paar zinnen uit waarom jouw maat voor "foutpercentage" echt ergens op slaat, en het een goede indicator geeft. 
5. Doe tenslotte hetzelfde voor `edges`. 
    * Hier doe je natuurlijk een `merge`
    * Bekijk hier eens het verschil tussen een outer en een inner join.
    * Leg uit wat er hier aan de hand is.
    * Kan je het verschil verklaren?


In [156]:
# vergelijk stats met dataset_stats_wikipedia

samen= pd.concat([dataset_stats_wikipedia, stats], axis=1)
samen.head(10)
#def fout_percentage(col1,col2,df):
     # jouw code hier 

 

#for col in stats.columns:
#    print "%s\t%s\t%s" %(col, fout_percentage(col, col+'my', samen)

,Language,Code,Edits,Editors,Avg. edits per editor,Edits by mulitlinguals,Multilinguals,Avg. edits per multilingual editor,Total editors,Total multilinguals,Editors,Edits,Multilingual,Edits by multilinguals
0,Abkhazian,abk,2807,15,187.133333,1494,7,213.428571,2562860,279172,NaN,NaN,NaN,NaN
1,Afar,aar,46,4,11.500000,9,3,3.000000,2562860,279172,NaN,NaN,NaN,NaN
2,Afrikaans,afr,103652,511,202.841487,95988,352,272.693182,2562860,279172,NaN,NaN,NaN,NaN
3,Akan,aka,327,31,10.548387,240,24,10.000000,2562860,279172,NaN,NaN,NaN,NaN
4,Albanian,sqi,196685,1996,98.539579,137800,1083,127.239150,2562860,279172,NaN,NaN,NaN,NaN
5,Amharic,amh,4596,68,67.588235,4039,45,89.755556,2562860,279172,NaN,NaN,NaN,NaN
6,Arabic,ara,2178719,18258,119.329554,655776,4647,141.118141,2562860,279172,NaN,NaN,NaN,NaN
7,Aragonese,arg,60924,141,432.085106,58810,103,570.970874,2562860,279172,NaN,NaN,NaN,NaN
8,Armenian,hye,65797,638,103.130094,29242,301,97.149502,2562860,279172,NaN,NaN,NaN,NaN
9,Arpitan,frp,8357,28,298.464286,8090,22,367.727273,2562860,279172,NaN,NaN,NaN,NaN


In [231]:
samen.describe()

,Editors,Edits,Multilinguals,Edits by mulitlinguals,Editorsmy,Editsmy,Multilingualsmy,Edits by mulitlingualsmy
count,238.000000,2.380000e+02,238.000000,238.000000,238.000000,2.380000e+02,238.000000,238.000000
mean,12243.554622,1.608757e+06,2648.226891,729135.878151,12307.088235,1.608984e+06,2657.487395,729154.399160
std,105148.317861,1.322120e+07,15606.769464,5342618.270859,105478.073646,1.322242e+07,15647.386691,5342696.119103
min,1.000000,2.000000e+00,1.000000,2.000000,1.000000,2.000000e+00,1.000000,2.000000
25%,20.250000,1.410500e+03,13.000000,562.500000,21.000000,1.413500e+03,13.000000,562.500000
50%,61.500000,8.092500e+03,39.000000,5325.500000,63.500000,8.094500e+03,39.000000,5325.500000
75%,458.250000,1.099002e+05,238.500000,64802.000000,463.250000,1.099082e+05,240.000000,64802.000000
max,1589250.000000,1.983610e+08,218320.000000,78341211.000000,1593939.000000,1.983783e+08,218823.000000,78342217.000000


In [ ]:
# vergelijk edges met "inner join"
 

In [ ]:
# vergelijk edges met "outer join"

edges2=pd.merge(edges,wikipedia_edges, how='outer', on=['SourceLanguageCode','TargetLanguageCode'])
print fout_percentage('Coocurrencesmy','Coocurrences', edges2)
edges2.describe()

# Compute our own PhiCorrelation and T-statistic

Maak een functie `PhiCorrelation(i,j)` die gegeven 2 talen, de $\phi$ correlatie en de $t$-statistic uitrekent volgens vergelijkingen [1] en [2] uit het artikel.

* Let vooral op hoe je $N$ bepaalt. En beschrijf goed waarom je precies die keuze hebt genomen. Je uitkomsten hangen daar sterk vanaf.

In [213]:
# Compute our own PhiCorrelation and T-statistic


def PhiCorrelation(i,j):
     #Your code here
    return Mij,Mi, Mj,  N, PhiCorrelation, Tstatistic

# test
PhiCorrelation('afr','nld')

(79, 531, 26706, 2929087, 0.019785094309992902, 3.2337861998200852)

# Reflectie
Je hebt getracht een deel van het artikel te reproduceren. Ook hebben we gedacht over het artikel en de mate van herhaalbaarheid van de vermelde gegevens. 

Geef nu in een half A4 je eigen mening. Behandel deze twee punten:

1. Wat was al goed gedaan?
2. Wat kon beter en hoe dan?